# Exploration package ARIMA

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from US_markets.arima import *

In [3]:
data = get_data()

In [ ]:
df_sym, train, test, date_test = preprocess(data, 'MSFT')

In [ ]:
df_sym

In [ ]:
train.shape, test.shape

In [ ]:
best_order = get_best_model(df_sym, train)

In [ ]:
best_order

In [ ]:
model = ARIMA(train, order=best_order).fit()

In [ ]:
len_test = len(test)

In [ ]:
forecast, stderr, conf_int = forecast(model, len_test)

In [ ]:
accuracy = forecast_accuracy(forecast, test)

In [ ]:
accuracy

In [ ]:
import matplotlib.pyplot as plt

forecast.index = test.index
conf_int.index = test.index

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(forecast, c='orange')
plt.plot(conf_int['low'], label='low', c='orange', ls='--')
plt.plot(conf_int['high'], label='high', c='orange', ls='--')
plt.plot(test, c='blue')
plt.legend()
plt.fill_between(test.index, conf_int['low'], conf_int['high'], color='k', alpha=.15)

In [ ]:
data2 = get_data()

In [ ]:
df_sym, train, test, date_test = preprocess(data2, 'MSFT')

In [ ]:
df_sym

In [4]:
data = data['symbol']
data

0           MSFT
1           MSFT
2           MSFT
3           MSFT
4           MSFT
            ... 
23528430    FATE
23528431    FATE
23528432    FATE
23528433    FATE
23528434    FATE
Name: symbol, Length: 23528435, dtype: object

In [7]:
data = data.unique()

In [8]:
data

array(['MSFT', 'AAPL', 'EBAY', ..., 'TSHA', 'OYST', 'SAQNW'], dtype=object)

In [20]:
tab_cmp = pd.DataFrame(data, columns=['symbol']).sort_values(by='symbol')

In [21]:
tab_cmp.to_csv('../raw_data/compagny.csv', encoding='utf-8', index=False)

In [1]:
import yfinance as yf

In [5]:
msft = yf.Ticker('AAPL')

In [6]:
msft.history(period="max")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100751,0.101189,0.100751,0.100751,469033600,0.0,0.0
1980-12-15,0.095933,0.095933,0.095495,0.095495,175884800,0.0,0.0
1980-12-16,0.088923,0.088923,0.088485,0.088485,105728000,0.0,0.0
1980-12-17,0.090676,0.091114,0.090676,0.090676,86441600,0.0,0.0
1980-12-18,0.093304,0.093742,0.093304,0.093304,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2021-06-08,126.599998,128.460007,126.209999,126.739998,74403800,0.0,0.0
2021-06-09,127.209999,127.750000,126.519997,127.129997,56877900,0.0,0.0
2021-06-10,127.019997,128.190002,125.940002,126.110001,71186400,0.0,0.0
